In [19]:
import pandas
import pandas as pd
import numpy as np
import os
import sqlalchemy
import pymysql
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline
import matplotlib.pyplot as plt

The data from the df below is the basis list of the stocks that I will be using the for built of the traded bot.

In [2]:
df = pd.read_csv("data/SP500Data.csv")

In [3]:
df.head()

,NUMBER,TICKER,NAME,MARKET CAP,STOCK PRICE,%CHANGE,REVENUE
0,1,AAPL,Apple Inc.,"3,915.30B",259.02,0.0032,391.04B
1,2,NVDA,NVIDIA Corporation,"3,426.89B",139.93,-0.0021,113.27B
2,3,MSFT,Microsoft Corporation,"3,257.30B",438.11,-0.0028,254.19B
3,4,GOOG,Alphabet Inc.,"2,408.66B",197.10,-0.0024,339.86B
4,5,GOOGL,Alphabet Inc.,"2,394.34B",195.60,-0.0026,339.86B


Below is the code for the creation of all the Excel files. I just added the Excel formula to each Excel file, and opening the app clicked enter for the formula to update (could have been done with another script.)

In [4]:
for tick in df["TICKER"]:
    dfTick = pd.DataFrame(['=STOCKHISTORY("%s", "1980-01-01", "2024-12-25",0,0,0,1,2,3,4,5)'%tick])
    ###dfTick.to_csv(path_or_buf="data/SP500DataSQL/%s.csv"%tick, header=False, index=False) ###DON'T RUN###
    break

Below I decided to add the column names and make the date column the 'index column'. A couple of problems I went through were the appearance of euro marks in the Excel files, also there were some problems with the regex and the column names being converted to floats, this happened because the column names were added multiple times, so I needed to go through the first Excel file which is basically a test/trial file and update again to erase the column names. the try and except clauses are just for weird symbols that might appear in the Excel files.
Day 4 - Now I had to reformat the date column to be a true date instead of an object, this took a lot of attempts to get right, I kept getting errors because sometimes headers were duplicated or index columns were added incorrectly. Now I don't need the try and except clauses which leads me to believe now the code is better written.
Day 10 or so - Finally was illuminated enough to be able to move the data to sql tables, now I will actually explore and clean it, then might decide to add some more columns with important data that the model can use to better identify trends.

In [29]:
url_object = sqlalchemy.URL.create(
    "mysql+pymysql",
    username="*",
    password="*",  # plain (unescaped) text
    host="*",
    database="*",
    )
eng = sqlalchemy.create_engine(url_object)

In [63]:
df = pd.DataFrame
directory = os.fsencode("data/SP500Data")
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    df = pd.read_csv("data/SP500Data/%s"%filename)
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", drop=True, inplace=True) #make sure to convert date to datetime so it can be recognized as such by pandas
    df = df.replace("[^.0-9]", "", regex=True).astype(float) #Here we use some regex to convert all columns to float values
    #Percentage return from previous close day
    df['Returns'] = df['Close'].pct_change().round(8)
    # Calculate weekly returns
    df['Weekly Return'] = ((df['Close'] - df['Close'].shift(7)) / 100).round(8)
    # Calculate monthly returns
    df['Monthly Return'] = ((df['Close'] - df['Close'].shift(30)) / 100).round(8)
    # Calculate the daily price changes
    df['Change'] = df['Close'].diff().round(6)
    # Separate the gains and losses
    df['Gain'] = np.where(df['Change'] > 0, df['Change'], 0).round(6) #if 'Change' > 0 then it is a gain
    df['Loss'] = np.where(df['Change'] < 0, -df['Change'], 0).round(6) #else it is a loss.
    # Calculate the average gains and losses
    df['Avg Gain'] = df['Gain'].rolling(window=14).mean().round(6)
    df['Avg Loss'] = df['Loss'].rolling(window=14).mean().round(6)
    # Calculate the Relative Strength (RS)
    df['RS'] = (df['Avg Gain'] / df['Avg Loss']).round(6)
    # Calculate the RSI
    df['RSI'] = (100 - (100 / (1 + df['RS']))).round(6)
    # Calculate the Simple Moving Average with a span of 5 days
    df['MA5'] = df['Close'].rolling(window=5).mean().round(6)
    # Calculate the Simple Moving Average with a span of 10 days
    df['MA10'] = df['Close'].rolling(window=10).mean().round(6)
    # Calculate the Simple Moving Average with a span of 20 days
    df['MA20'] = df['Close'].rolling(window=20).mean().round(6)
    # Calculate the Simple Moving Average with a span of 50 days
    df['MA50'] = df['Close'].rolling(window=50).mean().round(6)
    # Calculate the Simple Moving Average with a span of 200 days
    df['MA200'] = df['Close'].rolling(window=200).mean().round(6)
    # Calculate the EWMA with a span of 10 days
    df['EWMA10'] = df['Close'].ewm(span=10, adjust=False).mean().round(6)
    # Calculate the EWMA with a span of 12 days for MACD
    df['EWMA12'] = df['Close'].ewm(span=12, adjust=False).mean().round(6)
    # Calculate the EWMA with a span of 26 days for MACD
    df['EWMA26'] = df['Close'].ewm(span=26, adjust=False).mean().round(6)
    # Calculate the EWMA with a span of 50 days
    df['EWMA50'] = df['Close'].ewm(span=50, adjust=False).mean().round(6)
    # Calculate the EWMA with a span of 200 days
    df['EWMA200'] = df['Close'].ewm(span=200, adjust=False).mean().round(6)
    # Calculate the 20-day rolling volatility (standard deviation of returns)
    df['Volatility20'] = df['Returns'].rolling(window=20).std().round(6)
    # Calculate the 30-day rolling volatility
    df['Volatility30'] = df['Returns'].rolling(window=30).std().round(6)
    # Calculate the MACD line
    df['MACD'] = (df['EWMA12'] - df['EWMA26']).round(6)
    # Calculate the Signal line (9 periods EMA of the MACD line)
    df['Signal Line'] = (df['MACD'].ewm(span=9, adjust=False).mean()).round(6)
    # Calculate the MACD Histogram
    df['MACD Histogram'] = (df['MACD'] - df['Signal Line']).round(6)
    #Calculate Standard Dev for 20MA
    df["STD20"] = (df['Close'].rolling(window=20).std()).round(6)
    # Calculate the Upper and Lower Bollinger Bands
    df['Upper Band'] = (df['MA20'] + (2 * df['STD20'])).round(6)
    df['Lower Band'] = (df['MA20'] - (2 * df['STD20'])).round(6)
    # Calculate the typical price
    df['Typical Price'] = ((df['High'] + df['Low'] + df['Close']) / 3).round(6)
    # Calculate the cumulative typical price * volume
    df['Cumulative TPV'] = ((df['Typical Price'] * df['Volume']).cumsum()).round(6)
    # Calculate the cumulative volume
    df['Cumulative Volume'] = (df['Volume'].cumsum()).round(6)
    # Calculate the VWAP
    df['VWAP'] = (df['Cumulative TPV'] / df['Cumulative Volume']).round(6)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(how='any', inplace=True)
    #df = df[df.Returns != 0] #Value from day when there was no trade.
    # #price rises next day then buy
    df['TargetBuy'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

    #price drops next day then sell
    df['TargetSell'] = np.where(df['Close'].shift(-1) < df['Close'], 1, 0)
    filename = (filename[:-4]).lower()
    df.to_sql(name="%s"%filename, con=eng, if_exists="replace", index=True)
    #df.to_csv(path_or_buf="data/SP500Data/%s"%filename) Commented it out because I am doing sql now.
    break

Need to check returns, daily, weekly and monthly. Then find a way to only have 1's appear when I need to sell or buy.

In [49]:
temNP = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
for i in range(1, len(temNP)):
        if temNP[i] == 1:
            j = i + 1
            while j < len(temNP) and temNP[j] >= temNP[j - 1]:
                temNP[j] = 0
                j += 1


In [52]:
temNP[0:40]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

I need to discover how to analyze the sql using python/pandas/numpy etc. I might just do it on DataGrip.

In [35]:
df = pd.DataFrame
directory = os.fsencode('data/SP500Data')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    filename = (filename[:-4]).lower()
    df = pd.read_sql('SELECT MA5, MA20, RSI, TargetBuy, TargetSell FROM %s'%filename, con=eng)
    break


In [36]:
df.head()

,MA5,MA20,RSI,TargetBuy,TargetSell
0,41.218,35.1960,82.408381,0,1
1,41.648,35.9395,85.219277,0,1
2,41.244,36.6535,79.232383,1,0
3,41.312,37.4135,67.165413,0,1
4,41.076,38.0955,57.249666,0,1
